<h1 align="center">Using Dataset Classes in PyTorch</h1>

Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro

Em problemas de `Machine Learning` e `Deep Learning`, muito esforço é dedicado à preparação dos dados. Os dados geralmente são confusos e precisam ser pré-processados ​​antes de serem usados ​​para treinar um modelo. Se os dados não forem preparados corretamente, o modelo não poderá generalizar bem. Algumas das etapas comuns necessárias para o pré-processamento de dados incluem:

* <font color="red">Normalização de dados (`Data normalization`):</font> Isto inclui a normalização dos dados entre um intervalo de valores em um Dataset.

* <font color="red">Aumento de dados (`Data augmentation`):</font> Isso inclui a geração de novas amostras a partir das existentes, adicionando ruído ou mudanças nas features para torná-los mais diversos.

A preparação de dados é uma etapa crucial em qualquer pipeline de Machine Learning. O `PyTorch` traz muitos módulos, como `torchvision`, que fornece Datasets e classes de Dataset para facilitar a preparação dos dados.

Neste script, aprenderemos a como trabalhar com conjuntos de dados e transformações no `PyTorch` para que você possa criar suas próprias `classes de conjunto de dados personalizadas` e manipular os conjuntos de dados da maneira que desejar. Em particular, aprenderemos:

* Como criar uma classe de `dataset` simples e aplicar transformações a ela.

* Como criar transformações chamáveis ​​e aplicá-las ao objeto `dataset`.

* Como compor várias transformações em um objeto `dataset`.

Observe que aqui você jogará com conjuntos de dados simples para compreensão geral dos conceitos, enquanto na próxima parte deste script você terá a chance de trabalhar com objetos de `dataset` para imagens.

# Criando uma classe de conjunto de dados simples

Antes de começar, teremos que importar alguns pacotes antes de criar a classe `dataset`.

In [4]:
import torch
from torch.utils.data import Dataset
torch.manual_seed(42)


Importaremos a classe abstrata `Dataset` de `torch.utils.data`. Portanto, substituímos os métodos abaixo na classe dataset:

* `__len__` para que `len(dataset)` possa nos dizer o tamanho do dataset.

* `__getitem__` para acessar as amostras de dados no dataset suportando a operação de indexação. <font color="yellow">Por exemplo:</font> `dataset[i]` pode ser usado para recuperar a i-ésima amostra de dados.

Da mesma forma, `torch.manual_seed()` força a função aleatória a produzir o mesmo número toda vez que é recompilada.

Agora, vamos definir a classe dataset:

In [5]:
class SimpleDataset(Dataset):
    # defining values in the constructor
    def __init__(self, data_length = 20, transform = None):
        self.x = 3 * torch.eye(data_length, 2)
        self.y = torch.eye(data_length, 4)
        self.transform = transform
        self.len = data_length
     
    # Getting the data samples
    def __getitem__(self, idx):
        sample = self.x[idx], self.y[idx]
        if self.transform:
            sample = self.transform(sample)     
        return sample
    
    # Getting data size/length
    def __len__(self):
        return self.len

No objeto construtor, criamos os valores das features e targets, ou seja , $x$ e $y$, atribuindo seus valores aos tensores `self.x` e `self.y`. Cada tensor carrega $20$ amostras de dados enquanto o atributo `data_length` armazena o número de amostras de dados. Vamos discutir sobre as transformações mais adiante.

O comportamento do objeto `SimpleDataset` é como qualquer iterável do `Python`, como uma lista ou uma tupla. Agora, vamos criar o objeto `SimpleDataset` e observar seu comprimento total e o valor no índice $1$.

In [6]:
dataset = SimpleDataset()

print("length of the SimpleDataset object: ", len(dataset))
print("accessing value at index 1 of the simple_dataset object: ", dataset[1])

length of the SimpleDataset object:  20
accessing value at index 1 of the simple_dataset object:  (tensor([0., 3.]), tensor([0., 1., 0., 0.]))


Como nosso conjunto de dados é iterável, vamos imprimir os quatro primeiros elementos usando um loop:

In [8]:
for i in range(4):
    x, y = dataset[i]
    print(x, y)

tensor([3., 0.]) tensor([1., 0., 0., 0.])
tensor([0., 3.]) tensor([0., 1., 0., 0.])
tensor([0., 0.]) tensor([0., 0., 1., 0.])
tensor([0., 0.]) tensor([0., 0., 0., 1.])


# Criando transformações chamáveis (Creating Callable Transforms)

Em vários casos, você precisará criar transformações chamáveis ​​para `normalizar` ou `padronizar` os dados. Essas transformações podem então ser aplicadas aos `tensores`. Vamos criar uma transformação chamável e aplicá-la ao nosso objeto `"dataset simples"` que criamos anteriormente.

In [9]:
# Creating a callable transform class mult_divide
class MultDivide:
    # Constructor
    def __init__(self, mult_x = 2, divide_y = 3):
        self.mult_x = mult_x
        self.divide_y = divide_y

    # caller
    def __call__(self, sample):
        x = sample[0]
        y = sample[1]
        x = x * self.mult_x
        y =y / self.divide_y
        sample = x, y
        return sample


Criamos uma transformação customizada simples `MultDivide` que multiplica $x$ com $2$ e divide $y$ por $3$. Isso não é para uso prático, mas para demonstrar como uma classe chamável pode funcionar como uma transformação para nossa classe `dataset`. Lembre-se, declaramos um parâmetro `transform = None` no `simple_dataset`. Agora, podemos substituí-lo pelo objeto de transformação personalizado com `None` que acabamos de criar. 


Então, vamos demonstrar como isso é feito e chamar esse objeto de transformação em dataset para ver como ele transforma os quatro primeiros elementos de nosso dataset.

In [10]:
# calling the transform object
mul_div = MultDivide()
custom_dataset = SimpleDataset(transform = mul_div)
 
for i in range(4):
    x, y = dataset[i]
    print('Idx: ', i, 'Original_x: ', x, 'Original_y: ', y)
    x_, y_ = custom_dataset[i]
    print('Idx: ', i, 'Transformed_x:', x_, 'Transformed_y:', y_)

Idx:  0 Original_x:  tensor([3., 0.]) Original_y:  tensor([1., 0., 0., 0.])
Idx:  0 Transformed_x: tensor([6., 0.]) Transformed_y: tensor([0.3333, 0.0000, 0.0000, 0.0000])
Idx:  1 Original_x:  tensor([0., 3.]) Original_y:  tensor([0., 1., 0., 0.])
Idx:  1 Transformed_x: tensor([0., 6.]) Transformed_y: tensor([0.0000, 0.3333, 0.0000, 0.0000])
Idx:  2 Original_x:  tensor([0., 0.]) Original_y:  tensor([0., 0., 1., 0.])
Idx:  2 Transformed_x: tensor([0., 0.]) Transformed_y: tensor([0.0000, 0.0000, 0.3333, 0.0000])
Idx:  3 Original_x:  tensor([0., 0.]) Original_y:  tensor([0., 0., 0., 1.])
Idx:  3 Transformed_x: tensor([0., 0.]) Transformed_y: tensor([0.0000, 0.0000, 0.0000, 0.3333])


Como você pode ver, a transformação foi aplicada com sucesso aos primeiros quatro elementos do conjunto de dados.

# Compondo várias transformações para conjuntos de dados (Composing Multiple Transforms for Datasets)

Muitas vezes, gostaríamos de realizar várias transformações em série em um conjunto de dados. Isso pode ser feito importando a classe `Compose` do módulo de transformações no `torchvision`. Por exemplo, digamos que construímos outra transformação `SubtractOne` e a aplicamos ao nosso conjunto de dados, além da transformação `MultDivide` que criamos anteriormente.

Depois de aplicada, a transformação recém-criada subtrairá $1$ de cada elemento do conjunto de dados.

In [11]:
from torchvision import transforms
 
# Creating subtract_one tranform
class SubtractOne:
    # Constructor
    def __init__(self, number = 1):
        self.number = number
        
    # caller
    def __call__(self, sample):
        x = sample[0]
        y = sample[1]
        x = x - self.number
        y = y - self.number
        sample = x, y
        return sample


Conforme especificado anteriormente, agora combinaremos ambas as transformações com o método `Compose`.

In [12]:
# Composing multiple transforms
mult_transforms = transforms.Compose([MultDivide(), SubtractOne()])

Observe que a primeira transformação `MultDivide` será aplicada no conjunto de dados e, em seguida, a transformação `SubtractOne` será aplicada nos elementos transformados do conjunto de dados.
Passaremos o objeto `Compose` (que contém a combinação de ambas as transformações: `MultDivide()` e `SubtractOne()`) para o nosso objeto `SimpleDataset`.

In [13]:
# Creating a new simple_dataset object with multiple transforms
new_dataset = SimpleDataset(transform = mult_transforms)


Agora que a combinação de múltiplas transformações foi aplicada ao conjunto de dados, vamos imprimir os primeiros quatro elementos do nosso conjunto de dados transformado.

In [14]:
for i in range(4):
    x, y = dataset[i]
    print('Idx: ', i, 'Original_x: ', x, 'Original_y: ', y)
    x_, y_ = new_dataset[i]
    print('Idx: ', i, 'Transformed x_:', x_, 'Transformed y_:', y_)
    

Idx:  0 Original_x:  tensor([3., 0.]) Original_y:  tensor([1., 0., 0., 0.])
Idx:  0 Transformed x_: tensor([ 5., -1.]) Transformed y_: tensor([-0.6667, -1.0000, -1.0000, -1.0000])
Idx:  1 Original_x:  tensor([0., 3.]) Original_y:  tensor([0., 1., 0., 0.])
Idx:  1 Transformed x_: tensor([-1.,  5.]) Transformed y_: tensor([-1.0000, -0.6667, -1.0000, -1.0000])
Idx:  2 Original_x:  tensor([0., 0.]) Original_y:  tensor([0., 0., 1., 0.])
Idx:  2 Transformed x_: tensor([-1., -1.]) Transformed y_: tensor([-1.0000, -1.0000, -0.6667, -1.0000])
Idx:  3 Original_x:  tensor([0., 0.]) Original_y:  tensor([0., 0., 0., 1.])
Idx:  3 Transformed x_: tensor([-1., -1.]) Transformed y_: tensor([-1.0000, -1.0000, -1.0000, -0.6667])


Juntando tudo, o código completo é o seguinte:

In [15]:
import torch
from torch.utils.data import Dataset
from torchvision import transforms
 
torch.manual_seed(2)
 
class SimpleDataset(Dataset):
    # defining values in the constructor
    def __init__(self, data_length = 20, transform = None):
        self.x = 3 * torch.eye(data_length, 2)
        self.y = torch.eye(data_length, 4)
        self.transform = transform
        self.len = data_length
     
    # Getting the data samples
    def __getitem__(self, idx):
        sample = self.x[idx], self.y[idx]
        if self.transform:
            sample = self.transform(sample)     
        return sample
    
    # Getting data size/length
    def __len__(self):
        return self.len
 
# Creating a callable tranform class mult_divide
class MultDivide:
    # Constructor
    def __init__(self, mult_x = 2, divide_y = 3):
        self.mult_x = mult_x
        self.divide_y = divide_y
    
    # caller
    def __call__(self, sample):
        x = sample[0]
        y = sample[1]
        x = x * self.mult_x
        y = y / self.divide_y
        sample = x, y
        return sample
 
# Creating subtract_one tranform
class SubtractOne:
    # Constructor
    def __init__(self, number = 1):
        self.number = number
        
    # caller
    def __call__(self, sample):
        x = sample[0]
        y = sample[1]
        x = x - self.number
        y = y - self.number
        sample = x, y
        return sample
 
# Composing multiple transforms
mult_transforms = transforms.Compose([MultDivide(), SubtractOne()])
 
# Creating a new simple_dataset object with multiple transforms
dataset = SimpleDataset()
new_dataset = SimpleDataset(transform = mult_transforms)
 
print("length of the simple_dataset object: ", len(dataset))
print("accessing value at index 1 of the simple_dataset object: ", dataset[1])
 
for i in range(4):
    x, y = dataset[i]
    print('Idx: ', i, 'Original_x: ', x, 'Original_y: ', y)
    x_, y_ = new_dataset[i]
    print('Idx: ', i, 'Transformed x_:', x_, 'Transformed y_:', y_)

length of the simple_dataset object:  20
accessing value at index 1 of the simple_dataset object:  (tensor([0., 3.]), tensor([0., 1., 0., 0.]))
Idx:  0 Original_x:  tensor([3., 0.]) Original_y:  tensor([1., 0., 0., 0.])
Idx:  0 Transformed x_: tensor([ 5., -1.]) Transformed y_: tensor([-0.6667, -1.0000, -1.0000, -1.0000])
Idx:  1 Original_x:  tensor([0., 3.]) Original_y:  tensor([0., 1., 0., 0.])
Idx:  1 Transformed x_: tensor([-1.,  5.]) Transformed y_: tensor([-1.0000, -0.6667, -1.0000, -1.0000])
Idx:  2 Original_x:  tensor([0., 0.]) Original_y:  tensor([0., 0., 1., 0.])
Idx:  2 Transformed x_: tensor([-1., -1.]) Transformed y_: tensor([-1.0000, -1.0000, -0.6667, -1.0000])
Idx:  3 Original_x:  tensor([0., 0.]) Original_y:  tensor([0., 0., 0., 1.])
Idx:  3 Transformed x_: tensor([-1., -1.]) Transformed y_: tensor([-1.0000, -1.0000, -1.0000, -0.6667])
